# AIM: Extract features on labeled data using the pretrained EEGNet
across subject pretext task

In [1]:
import numpy as np
import pandas as pd
import mne
import lightning.pytorch as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchmetrics import F1Score, Accuracy
import random
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from IPython.display import display
%matplotlib inline


# prevent extensive logging
mne.set_log_level('WARNING')

## Loading epoch data & participant data of labeled sample
These dataframes have been filtered and stored in a previous project. See https://github.com/TSmolders/Internship_EEG for original code

In [2]:

df_participants = pd.read_pickle(r'D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN_participants_V2_data\df_participants.pkl')
sample_df = pd.read_pickle(r'D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TD-BRAIN_extracted_features\df_selected_stat_features.pkl')
sample_ids = sample_df['ID'].unique() # obtain unique IDs from subsampled dataframe containing epoched features
df_sample = df_participants[df_participants['participants_ID'].isin(sample_ids)] # filter participants dataframe to only include subsampled IDs
df_sample = df_sample[df_sample['sessID'] == 1] # filter first session
print(df_sample.shape)
print(df_sample['diagnosis'].value_counts())



(225, 12)
diagnosis
ADHD       45
HEALTHY    45
MDD        45
OCD        45
SMC        45
Name: count, dtype: int64


In [3]:
# functions for loading the epoched EEG data
def get_filepath(epoch_dir, participant_ids):
    """
    Function to get the filepath of the epoched EEG recording
    :param epoch_dir: directory containing the epoched EEG recordings
    :param participant_ids: list of participant IDs to include
    """
    filepaths = []
    found_ids = set()
    
    for subdir, dirs, files in os.walk(epoch_dir):
        for file in files:
            for participant_id in participant_ids:
                if participant_id in file:
                    filepaths.append(os.path.join(subdir, file))
                    found_ids.add(participant_id)
    
    # Print participant IDs if no files matching those IDs are found
    for participant_id in participant_ids:
        if participant_id not in found_ids:
            print(f"No files found for participant ID {participant_id}")

    return filepaths

class EpochDataset(torch.utils.data.Dataset):
    def __init__(self, participant_ids, epoch_dir):
        self.filepaths = get_filepath(epoch_dir, participant_ids)
        self.participant_ids = participant_ids
        self.epochs = []
        self.participant_ids = []
        self._load_data()
        print(f"Number of epochs: {self.epochs.shape[0]}")
        print(f"Number of participants: {len(self.participant_ids)}")

    def _load_data(self):
        all_epochs = []
        for filepath in self.filepaths:
            epochs = torch.load(filepath)
            # get participant ID from filepath to make sure the participant ID is correct
            participant_id = filepath.split("\\")[-1].split(".")[0]
            all_epochs.append(epochs)
            self.participant_ids.extend([participant_id]*epochs.shape[0])
        self.epochs = np.concatenate(all_epochs, axis=0)

    def __len__(self):
        return self.epochs.shape[0]
    
    def __getitem__(self, idx):
        epoch = self.epochs[idx]
        participant_id = self.participant_ids[idx]
        return torch.tensor(epoch, dtype=torch.float32), participant_id

In [4]:
# load the epochs into a dataset
participant_ids = df_sample['participants_ID'].tolist()
dataset = EpochDataset(participant_ids, r"D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN-dataset-derivatives\thesis_epoched_data\EC")
print(len(dataset))
print(dataset[0][0].shape)
print(dataset[0][1])
print(dataset[1][1])

No files found for participant ID sub-88073521
Number of epochs: 2688
Number of participants: 2688
2688
torch.Size([26, 1244])
sub-87964717
sub-87964717


participant sub-88073521 EC's recording is marked as BAD by the preprocessing pipeline of TDBRAIN

## Transfering pretrained weights & extracting features

### Functions:

In [5]:
def transfer_weights(pretrained_weights, pretext_model):
    """
    Function to transfer the pretrained weights to the pretext model
    param: pretrained_weights: the weights to transfer in a dictionary
    param: pretext_model: the model to transfer the weights to
    """
    pretrained_model = pretext_model
    modified_keys = {}
    for k, v in pretrained_weights.items():
        decomposed_key = k.split('.')
        if decomposed_key[0] == 'EEGNet' or decomposed_key[0] == 'ShallowNet': # remove the first part of each key to match the model's keys
            pretrained_key = '.'.join(decomposed_key[1:])
            modified_keys[pretrained_key] = v

            
    pretrained_model.load_state_dict(modified_keys)
        
    return pretrained_model

def extract_features(pretrained_model, data, pretext_task, df_sample, to_disk=False, num_extracted_features=100, add_missing_ids=True):
    """
    Function to extract features from the pretrained model
    param: pretrained_model: the model to extract features from
    param: data: the dataset containing the epochs to extract features from
    param: pretext_task: a string indicating the specific pretext task to save the features as
    param: df_sample: the dataframe containing the sampled participant IDs and their corresponding diagnosis
    param: to_disk: boolean to save the features to disk
    param: num_extracted_features: the number of features to extract
    param: add_missing_ids: boolean to add missing participant IDs to the dataframe
    """
    dataloader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False)
    pretrained_model.eval()
    features_list = []
    participant_ids = []
    with torch.no_grad():  # Disable gradient calculation
        for batch in dataloader:
            epoch, participant_id = batch  # Remove the batch dimension
            epoch = epoch.unsqueeze(0)  # Add dimension
            # print(epoch.shape)
            features = pretrained_model(epoch)  # Extract features
            features = features.squeeze(0)
            features = features.numpy()
            features_list.append(features)
            participant_ids.append(participant_id[0])

    if add_missing_ids:
        # Probably not the best idea to add missing IDs with NaN values (in this
        # case only one ID), but the baselines from the previous internetship were 
        # trained with the same approach (using mean impute for the 12 missing epochs of one participant). 
        # Considering only 1 participant is missing (due to 1 BAD preprocessing file), 
        # it shouldn't have a large impact on the results. But it's something to keep in mind for future work.

        # check if any IDs are missing compared to the sampled IDs
        missing_ids = set(df_sample['participants_ID'].tolist()) - set(participant_ids)
        # if they are missing, add them with NaN values
        if missing_ids:
            for missing_id in missing_ids:
                num_missing_epochs = 12  # Assuming 12 missing epochs for each missing participant
                for _ in range(num_missing_epochs):
                    features_list.append([np.nan] * num_extracted_features)
                    participant_ids.append(missing_id)

    features_df = pd.DataFrame(features_list) # store as dataframe
    features_df['ID'] = participant_ids # add participant IDs to the dataframe

    # map the diagnosis values from df_sample to the dataframe based on participant IDs
    features_df['diagnosis'] = features_df['ID'].map(df_sample.set_index('participants_ID')['diagnosis'])

    
    print(f'{features_df.shape = }')
    # print(f'{features_df["diagnosis"].value_counts()}')
    # print(f'{features_df.isnull().sum()}')
    display(features_df.head(3))


    if to_disk:
        features_df.to_pickle(f'D:/Documents/Master_Data_Science/Thesis/thesis_code/DataScience_Thesis/data/SSL_features/df_{pretext_task}_features.pkl')
    
    return features_df

def evaluate_features(features_df):
    """
    Function to quickly evaluate the extracted features. Doesn't stratify/group data splitting! 
    """
    groups = features_df['ID']
    X = features_df.drop(['ID', 'diagnosis'], axis=1)
    y = features_df['diagnosis']

    # if X contains NaN values, impute them with the feature-wise mean
    if X.isnull().values.any():
        X = X.fillna(X.mean())

    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=False)

    # Get the train and test indices
    for train_index, test_index in sgkf.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        break  # Only use the first split
    
    # quick SVM model
    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('quick SVM model')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(f1_score(y_test, y_pred, average='macro'))
    print()

    # quick random forest model
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('quick random forest model')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(f1_score(y_test, y_pred, average='macro'))

    return

def get_ssl_features(
        pretext_task,
        data,
        df_sample,
        num_extracted_features=100,
        eval=True,
        to_disk=True,
        pretext_model='EEGNet',
        weights_dir=r'D:\Documents\Master_Data_Science\Thesis\thesis_code\DataScience_Thesis\data\pretext_model_weights'
                     ):
    """
    Obtains SSL features from the weights trained by the pretext model.
    param: pretext_task: a string indicating the specific pretext task to load the weights from and save the features as
    param: data: the dataset containing the epochs to extract features from
    param: df_sample: the dataframe containing the sampled participant IDs and their corresponding diagnosis
    param: num_extracted_features: the number of features to extract
    param: eval: boolean to evaluate the features
    param: to_disk: boolean to save the features to disk
    param: pretext_model: the model to extract features from
    param: weights_dir: the directory containing the weights of the pretext model
    """
    # Need to define model class here to avoid issues with different number of extracted features
    # create Conv2d with max norm constraint
    class Conv2dWithConstraint(nn.Conv2d):
        def __init__(self, *args, max_norm: int = 1, **kwargs):
            self.max_norm = max_norm
            super(Conv2dWithConstraint, self).__init__(*args, **kwargs)

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            self.weight.data = torch.renorm(self.weight.data, p=2, dim=0, maxnorm=self.max_norm)
            return super(Conv2dWithConstraint, self).forward(x)
        
    class EEGNet(nn.Module):
        """
        Code taken and adjusted from pytorch implementation of EEGNet
        url: https://github.com/torcheeg/torcheeg/blob/v1.1.0/torcheeg/models/cnn/eegnet.py#L5
        """
        def __init__(self,
                    chunk_size: int = 1244, # number of data points in each EEG chunk
                    num_electrodes: int = 26, # number of EEG electrodes
                    F1: int = 8, # number of filters in first convolutional layer
                    F2: int = 16, # number of filters in second convolutional layer
                    D: int = 2, # depth multiplier
                    num_extracted_features: int = num_extracted_features, # number of features to extract
                    kernel_1: int = 64, # the filter size of block 1 (half of sfreq (125 Hz))
                    kernel_2: int = 16, # the filter size of block 2 (one eight of sfreq (500 Hz))
                    dropout: float = 0.25): # dropout rate
            super(EEGNet, self).__init__()
            self.F1 = F1
            self.F2 = F2
            self.D = D
            self.chunk_size = chunk_size
            self.num_extracted_features = num_extracted_features
            self.num_electrodes = num_electrodes
            self.kernel_1 = kernel_1
            self.kernel_2 = kernel_2
            self.dropout = dropout

            self.block1 = nn.Sequential(
                nn.Conv2d(1, self.F1, (1, self.kernel_1), stride=1, padding=(0, self.kernel_1 // 2), bias=False),
                nn.BatchNorm2d(self.F1, momentum=0.01, affine=True, eps=1e-3),
                Conv2dWithConstraint(self.F1,
                                    self.F1 * self.D, (self.num_electrodes, 1),
                                    max_norm=1,
                                    stride=1,
                                    padding=(0, 0),
                                    groups=self.F1,
                                    bias=False), nn.BatchNorm2d(self.F1 * self.D, momentum=0.01, affine=True, eps=1e-3),
                nn.ELU(), nn.AvgPool2d((1, 4), stride=4), nn.Dropout(p=dropout))

            self.block2 = nn.Sequential(
                nn.Conv2d(self.F1 * self.D,
                        self.F1 * self.D, (1, self.kernel_2),
                        stride=1,
                        padding=(0, self.kernel_2 // 2),
                        bias=False,
                        groups=self.F1 * self.D),
                nn.Conv2d(self.F1 * self.D, self.F2, 1, padding=(0, 0), groups=1, bias=False, stride=1),
                nn.BatchNorm2d(self.F2, momentum=0.01, affine=True, eps=1e-3), nn.ELU(), nn.AvgPool2d((1, 8), stride=8),
                nn.Dropout(p=dropout))

            self.lin = nn.Linear(self.feature_dim(), num_extracted_features, bias=False)


        def feature_dim(self):
            # function to calculate the number of features after the convolutional blocks
            with torch.no_grad():
                mock_eeg = torch.zeros(1, 1, self.num_electrodes, self.chunk_size)

                mock_eeg = self.block1(mock_eeg)
                mock_eeg = self.block2(mock_eeg)

            return self.F2 * mock_eeg.shape[3]

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            x = self.block1(x)
            x = self.block2(x)
            x = x.flatten(start_dim=1)
            x = self.lin(x)
            return x
    
    class ShallowNet(nn.Module):
        """
        Pytorch implementation of the ShallowNet Encoder.
        Code taken and adjusted from:
        https://github.com/MedMaxLab/selfEEG/blob/024402ba4bde95051d86ab2524cc71105bfd5c25/selfeeg/models/zoo.py#L693
        """

        def __init__(self,
                    samples=1244,
                    chans=26, # number of EEG channels
                    F=40, # number of output filters in the temporal convolution layer
                    K1=25, # length of the temporal convolutional layer
                    pool=75, # temporal pooling kernel size
                    dropout=0.2, # dropout probability
                    num_extracted_features=num_extracted_features # number of features to extract
                    ):

            super(ShallowNet, self).__init__()
            self.conv1 = nn.Conv2d(1, F, (1, K1), stride=(1, 1))
            self.conv2 = nn.Conv2d(F, F, (chans, 1), stride=(1, 1))
            self.batch1 = nn.BatchNorm2d(F)
            self.pool2 = nn.AvgPool2d((1, pool), stride=(1, 15))
            self.flatten2 = nn.Flatten()
            self.drop1 = nn.Dropout(dropout)
            self.lin = nn.Linear(
                F * ((samples - K1 + 1 - pool) // 15 + 1), num_extracted_features
            )

        def forward(self, x):
            x = self.conv1(x)
            x = self.conv2(x)
            x = self.batch1(x)
            x = torch.square(x)
            x = self.pool2(x)
            x = torch.log(torch.clamp(x, 1e-7, 10000))
            x = self.flatten2(x)
            x = self.drop1(x)
            x = self.lin(x)

            return x
    
    if pretext_model == 'EEGNet':
        pretext_model = EEGNet()
    if pretext_model == 'ShallowNet':
        pretext_model = ShallowNet()
        
    pretrained_weights = torch.load(f'{weights_dir}\{pretext_task}_weights.pt')
    pretrained_model = transfer_weights(pretrained_weights, pretext_model)
    features_df = extract_features(pretrained_model, data, pretext_task, df_sample=df_sample, to_disk=to_disk, num_extracted_features=num_extracted_features)
    if eval:
        evaluate_features(features_df)
    
    return

### 0. randomly initialized model

In [7]:
features_df = extract_features(EEGNet(), dataset, df_sample=df_sample, pretext_task='random', to_disk=False)
evaluate_features(features_df)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-0.025231,0.558265,0.386430,-0.427448,-0.195549,-0.385354,0.035971,0.385304,0.106288,0.573986,...,0.261263,0.668248,-0.518208,0.242272,0.186938,0.214983,0.118263,-0.321973,sub-87964717,SMC
1,-0.111984,0.977839,0.131731,-0.051678,-0.290045,-0.434035,0.237411,-0.101681,-0.275212,0.762277,...,0.520117,0.624576,-0.485054,0.251544,-0.037080,-0.125105,-0.325259,-0.672581,sub-87964717,SMC
2,-0.033515,0.690026,-0.037421,-0.427968,-0.044509,-0.273484,0.223849,0.226518,0.001657,0.772421,...,0.178688,0.269243,-0.326347,-0.047144,-0.056313,0.331418,0.099657,-0.459115,sub-87964717,SMC


quick SVM model
[[58 17  5 16 12]
 [15 36  9 32 16]
 [13 29 24 21 21]
 [ 0 27 21 33 27]
 [ 3  4 15 26 60]]
              precision    recall  f1-score   support

        ADHD       0.65      0.54      0.59       108
     HEALTHY       0.32      0.33      0.33       108
         MDD       0.32      0.22      0.26       108
         OCD       0.26      0.31      0.28       108
         SMC       0.44      0.56      0.49       108

    accuracy                           0.39       540
   macro avg       0.40      0.39      0.39       540
weighted avg       0.40      0.39      0.39       540

0.3899649803774411

quick random forest model
[[62 14 14 10  8]
 [18 34 20 22 14]
 [22 28 26 15 17]
 [10 24 28 18 28]
 [ 3 23 15 19 48]]
              precision    recall  f1-score   support

        ADHD       0.54      0.57      0.56       108
     HEALTHY       0.28      0.31      0.29       108
         MDD       0.25      0.24      0.25       108
         OCD       0.21      0.17      0.19       

### 1. Within RP pretext task

#### 1a. Tpos = 1

##### ShallowNet (100 extracted features, 0.25 dropout, 0 weight decay)

In [6]:
# best model checkpoint
get_ssl_features('withinRP_tpos1_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-7.873107,15.091286,-7.697298,13.202025,-4.282566,-3.318047,-36.847160,22.292902,2.501897,-1.045815,...,-24.234993,-7.720624,-13.798555,-8.757239,12.269991,3.054723,42.745960,13.857347,sub-87964717,SMC
1,-0.729304,17.087202,2.456885,1.755869,-1.507568,10.947352,-20.794769,12.217931,-3.787356,-9.726949,...,-23.778820,-11.793024,-6.711127,-29.406527,4.221903,-8.824801,14.893806,33.437538,sub-87964717,SMC
2,-10.777116,-9.978358,-9.289070,1.351083,1.979544,5.139612,-41.853619,19.017334,4.709233,2.246111,...,-13.810199,-1.916366,-6.284012,-5.135130,24.972021,5.618553,28.810680,17.780787,sub-87964717,SMC


quick SVM model
[[31 16 30 28  3]
 [15 56  9 26  2]
 [ 2 22 17 41 26]
 [17  3 42 30 16]
 [13 11 18 16 50]]
              precision    recall  f1-score   support

        ADHD       0.40      0.29      0.33       108
     HEALTHY       0.52      0.52      0.52       108
         MDD       0.15      0.16      0.15       108
         OCD       0.21      0.28      0.24       108
         SMC       0.52      0.46      0.49       108

    accuracy                           0.34       540
   macro avg       0.36      0.34      0.35       540
weighted avg       0.36      0.34      0.35       540

0.3464812599216066

quick random forest model
[[44 13 38 11  2]
 [11 54  9 23 11]
 [10 22 16 30 30]
 [21 17 27 27 16]
 [15 14 22 11 46]]
              precision    recall  f1-score   support

        ADHD       0.44      0.41      0.42       108
     HEALTHY       0.45      0.50      0.47       108
         MDD       0.14      0.15      0.15       108
         OCD       0.26      0.25      0.26       

In [7]:
# overtrained model
get_ssl_features('fullytrained_withinRP_tpos1_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-11.957186,20.186932,-14.498624,14.704771,-9.023864,-3.734034,-42.590805,21.903591,-3.046731,-1.113170,...,-31.541496,-4.592828,-12.270212,-7.265633,18.689177,3.028424,45.246243,11.589210,sub-87964717,SMC
1,-5.648727,23.456623,-2.978145,0.892450,-8.329612,7.947573,-26.640871,17.140015,-9.700799,-11.252666,...,-27.217875,-7.679440,-6.539578,-26.954227,7.023908,-10.930799,16.495628,29.650238,sub-87964717,SMC
2,-14.221167,-2.145764,-13.633868,2.644824,-0.934855,3.786961,-47.581909,22.221910,1.091361,2.606711,...,-17.584354,5.507090,-2.077388,0.382738,28.874641,4.333996,31.933163,10.161799,sub-87964717,SMC


quick SVM model
[[31 19 31 24  3]
 [12 53 11 30  2]
 [ 1 15 17 44 31]
 [18  8 39 27 16]
 [15 11 21 16 45]]
              precision    recall  f1-score   support

        ADHD       0.40      0.29      0.34       108
     HEALTHY       0.50      0.49      0.50       108
         MDD       0.14      0.16      0.15       108
         OCD       0.19      0.25      0.22       108
         SMC       0.46      0.42      0.44       108

    accuracy                           0.32       540
   macro avg       0.34      0.32      0.33       540
weighted avg       0.34      0.32      0.33       540

0.3272267667490227

quick random forest model
[[34 14 37 19  4]
 [13 52 10 28  5]
 [ 8 20 16 31 33]
 [17 14 26 35 16]
 [13 15 24 16 40]]
              precision    recall  f1-score   support

        ADHD       0.40      0.31      0.35       108
     HEALTHY       0.45      0.48      0.47       108
         MDD       0.14      0.15      0.14       108
         OCD       0.27      0.32      0.30       

#### 1b. Tpos = 2

##### ShallowNet (100 extracted features, 0.25 dropout, 0 weight decay)

In [8]:
# best model checkpoint
get_ssl_features('withinRP_tpos2_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-2.800590,-20.537098,1.041329,-20.300411,14.366090,-13.551474,6.803631,-2.455139,19.073044,18.811035,...,4.933003,-3.924602,-35.656178,-13.590713,-92.758011,20.071548,4.622444,42.527649,sub-87964717,SMC
1,9.321916,-19.830301,39.421005,8.528296,9.487025,-4.109882,-4.919022,-32.817654,2.888638,2.256889,...,-10.497651,-1.762688,-27.943745,2.880476,-90.966583,27.967663,-11.685765,56.895626,sub-87964717,SMC
2,-4.721153,-12.858103,-18.349129,2.207919,30.092438,8.257747,-8.993794,-8.988847,9.059566,17.941257,...,8.123940,1.422324,-19.561935,18.007292,-88.497719,3.182600,7.131669,36.671753,sub-87964717,SMC


quick SVM model
[[40  9 21 33  5]
 [13 57  9 23  6]
 [13 18 13 39 25]
 [28 13 35 24  8]
 [13  2 21 19 53]]
              precision    recall  f1-score   support

        ADHD       0.37      0.37      0.37       108
     HEALTHY       0.58      0.53      0.55       108
         MDD       0.13      0.12      0.13       108
         OCD       0.17      0.22      0.20       108
         SMC       0.55      0.49      0.52       108

    accuracy                           0.35       540
   macro avg       0.36      0.35      0.35       540
weighted avg       0.36      0.35      0.35       540

0.3521233295244985

quick random forest model
[[39  7 28 26  8]
 [15 58  9 17  9]
 [ 9 13 22 30 34]
 [28 15 31 25  9]
 [11  6 27 12 52]]
              precision    recall  f1-score   support

        ADHD       0.38      0.36      0.37       108
     HEALTHY       0.59      0.54      0.56       108
         MDD       0.19      0.20      0.20       108
         OCD       0.23      0.23      0.23       

In [9]:
# overtrained model
get_ssl_features('fullytrained_withinRP_tpos2_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-5.350096,-16.341528,11.377154,-14.873400,4.859892,-9.365457,4.390083,-7.269978,16.891697,12.534257,...,0.871874,1.503397,-29.794420,-9.971622,-80.672577,5.152561,1.807938,45.719608,sub-87964717,SMC
1,7.768473,-16.740202,47.974819,13.946201,1.879747,-1.673598,-9.230963,-40.030296,2.627961,-4.795707,...,-14.857118,6.100892,-20.972950,5.828097,-76.341156,15.329967,-9.053797,64.190437,sub-87964717,SMC
2,-5.174697,-8.114922,-12.164124,7.423402,23.240135,14.174348,-10.055016,-16.107849,4.975190,8.302020,...,7.647080,4.383949,-13.693792,25.971663,-78.651260,-8.539501,9.350345,40.553997,sub-87964717,SMC


quick SVM model
[[40  7 22 33  6]
 [14 57  8 23  6]
 [11 17 15 38 27]
 [28 13 35 26  6]
 [15  3 22 13 55]]
              precision    recall  f1-score   support

        ADHD       0.37      0.37      0.37       108
     HEALTHY       0.59      0.53      0.56       108
         MDD       0.15      0.14      0.14       108
         OCD       0.20      0.24      0.22       108
         SMC       0.55      0.51      0.53       108

    accuracy                           0.36       540
   macro avg       0.37      0.36      0.36       540
weighted avg       0.37      0.36      0.36       540

0.3627877725808097

quick random forest model
[[35  6 28 31  8]
 [12 52 10 27  7]
 [11 17 16 34 30]
 [30 12 25 32  9]
 [13  3 23 13 56]]
              precision    recall  f1-score   support

        ADHD       0.35      0.32      0.33       108
     HEALTHY       0.58      0.48      0.53       108
         MDD       0.16      0.15      0.15       108
         OCD       0.23      0.30      0.26       

#### 1c. Tpos = 4

##### ShallowNet (100 extracted features, 0.25 dropout, 0 weight decay)

In [10]:
# best model checkpoint
get_ssl_features('withinRP_tpos4_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,11.287477,9.722813,-27.271212,-3.181419,-30.460697,-33.613594,0.961078,8.589078,-14.887156,10.084439,...,-42.411800,-51.050823,16.006048,37.540501,43.915646,2.213799,28.196299,-1.963492,sub-87964717,SMC
1,5.513151,-12.670759,-23.390087,6.053120,-18.902119,-36.829834,3.659711,-2.479099,2.730038,-15.643980,...,-25.019787,-33.078655,0.774869,39.382778,49.675468,-13.885685,28.928642,-28.355898,sub-87964717,SMC
2,5.281176,-20.300100,-25.288694,16.631395,-30.954985,-33.334126,-7.639419,16.125885,2.911843,-2.006031,...,-9.145327,-28.068089,-8.121552,26.115807,43.202961,-7.343721,29.241407,-12.042986,sub-87964717,SMC


quick SVM model
[[34  7 30 33  4]
 [13 51  2 30 12]
 [10  6 24 34 34]
 [18  7 42 31 10]
 [17  4 23 11 53]]
              precision    recall  f1-score   support

        ADHD       0.37      0.31      0.34       108
     HEALTHY       0.68      0.47      0.56       108
         MDD       0.20      0.22      0.21       108
         OCD       0.22      0.29      0.25       108
         SMC       0.47      0.49      0.48       108

    accuracy                           0.36       540
   macro avg       0.39      0.36      0.37       540
weighted avg       0.39      0.36      0.37       540

0.36752683817573056

quick random forest model
[[37  5 38 24  4]
 [14 55  4 25 10]
 [10 18 22 19 39]
 [28 10 23 25 22]
 [10  7 26 15 50]]
              precision    recall  f1-score   support

        ADHD       0.37      0.34      0.36       108
     HEALTHY       0.58      0.51      0.54       108
         MDD       0.19      0.20      0.20       108
         OCD       0.23      0.23      0.23      

In [11]:
# overtrained model
get_ssl_features('fullytrained_withinRP_tpos4_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,11.509591,4.715483,-17.705515,-1.842910,-28.218607,-39.568161,-12.195035,9.150964,-16.138863,12.646578,...,-38.250252,-41.804905,23.515955,36.385975,47.415794,7.955133,24.904104,-2.655988,sub-87964717,SMC
1,4.748695,-16.621571,-15.363754,7.188796,-14.818498,-41.966766,-2.576510,-8.475674,1.363800,-17.907534,...,-26.020943,-25.372997,2.100488,40.965748,52.628201,-5.850641,29.081211,-31.326591,sub-87964717,SMC
2,7.502552,-23.221539,-20.054428,19.934536,-29.511770,-40.836624,-16.619963,11.372686,5.875911,-2.577754,...,-6.975539,-18.802704,-6.180090,24.542673,45.504395,4.344173,26.664701,-16.836561,sub-87964717,SMC


quick SVM model
[[33  8 31 33  3]
 [10 58  5 24 11]
 [ 7  7 26 36 32]
 [13  7 48 30 10]
 [17  6 21 10 54]]
              precision    recall  f1-score   support

        ADHD       0.41      0.31      0.35       108
     HEALTHY       0.67      0.54      0.60       108
         MDD       0.20      0.24      0.22       108
         OCD       0.23      0.28      0.25       108
         SMC       0.49      0.50      0.50       108

    accuracy                           0.37       540
   macro avg       0.40      0.37      0.38       540
weighted avg       0.40      0.37      0.38       540

0.38219013910256205

quick random forest model
[[34  6 27 31 10]
 [11 53  5 25 14]
 [14  9 19 25 41]
 [26 12 34 19 17]
 [15 11 18  9 55]]
              precision    recall  f1-score   support

        ADHD       0.34      0.31      0.33       108
     HEALTHY       0.58      0.49      0.53       108
         MDD       0.18      0.18      0.18       108
         OCD       0.17      0.18      0.18      

### 2. Across RP pretext task

#### 2a. Tpos = 1

In [12]:
# best model checkpoint
get_ssl_features('acrossRP_tpos1_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.752588,-2.037266,-0.892866,-2.983312,3.480185,0.197868,-4.461242,2.704291,5.768301,-1.427948,...,-3.124218,0.433291,3.101932,-0.421450,2.292037,-0.859624,-1.372767,-0.891320,sub-87964717,SMC
1,0.580794,-2.003693,-1.217485,-3.164376,3.833876,-0.040860,-4.348482,2.570293,6.199616,-1.754639,...,-3.607905,0.007110,2.586107,-0.169564,2.800902,-0.160662,-1.024956,-0.976112,sub-87964717,SMC
2,0.100397,-1.433448,-1.182816,-3.156327,3.642837,0.099968,-4.493185,2.230083,5.910032,-1.961049,...,-3.464329,0.054942,2.445100,0.178928,2.547223,-0.250433,-0.846545,-0.710079,sub-87964717,SMC


quick SVM model
[[24  8 50 12 14]
 [16  1 22 65  4]
 [11  4  4 61 28]
 [14 13 17 47 17]
 [10  2 51 18 27]]
              precision    recall  f1-score   support

        ADHD       0.32      0.22      0.26       108
     HEALTHY       0.04      0.01      0.01       108
         MDD       0.03      0.04      0.03       108
         OCD       0.23      0.44      0.30       108
         SMC       0.30      0.25      0.27       108

    accuracy                           0.19       540
   macro avg       0.18      0.19      0.18       540
weighted avg       0.18      0.19      0.18       540

0.17674501453039593

quick random forest model
[[46 13 38  6  5]
 [24 35 20 22  7]
 [12 31 10 30 25]
 [18 37 13 27 13]
 [17 13 42 12 24]]
              precision    recall  f1-score   support

        ADHD       0.39      0.43      0.41       108
     HEALTHY       0.27      0.32      0.30       108
         MDD       0.08      0.09      0.09       108
         OCD       0.28      0.25      0.26      

In [13]:
# overtrained model
get_ssl_features('fullytrained_acrossRP_tpos1_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,2.440967,-4.232209,-0.612363,-3.174919,-1.909258,-0.868210,1.429765,1.353098,7.891938,4.612187,...,5.544913,7.499822,-3.835550,-0.862276,-0.210844,4.913355,-1.792756,-2.186712,sub-87964717,SMC
1,2.970114,-0.499616,-7.039492,-1.730707,0.673835,-2.138123,-2.780345,-4.363658,8.745045,0.866436,...,5.733691,-0.358436,-5.889634,1.541517,6.986569,-1.031410,-1.118753,1.107490,sub-87964717,SMC
2,-3.917462,-1.091859,0.409602,-1.205054,3.303648,6.419760,-4.377750,-0.914693,5.748407,6.521739,...,5.484847,3.648763,-5.978112,-2.794250,-0.894398,-2.091547,-6.867914,4.316982,sub-87964717,SMC


quick SVM model
[[53 10 18 10 17]
 [21 31 12 30 14]
 [ 6 13  8 46 35]
 [ 5 11 21 48 23]
 [14  7 28 15 44]]
              precision    recall  f1-score   support

        ADHD       0.54      0.49      0.51       108
     HEALTHY       0.43      0.29      0.34       108
         MDD       0.09      0.07      0.08       108
         OCD       0.32      0.44      0.37       108
         SMC       0.33      0.41      0.37       108

    accuracy                           0.34       540
   macro avg       0.34      0.34      0.34       540
weighted avg       0.34      0.34      0.34       540

0.3354518210857226

quick random forest model
[[50 12 17 24  5]
 [15 20 18 31 24]
 [ 3 18 11 30 46]
 [10 19 13 42 24]
 [14 10 41 17 26]]
              precision    recall  f1-score   support

        ADHD       0.54      0.46      0.50       108
     HEALTHY       0.25      0.19      0.21       108
         MDD       0.11      0.10      0.11       108
         OCD       0.29      0.39      0.33       

#### 2b. Tpos = 2

In [14]:
# best model checkpoint
get_ssl_features('acrossRP_tpos2_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.289384,-3.968223,4.961030,0.585199,-3.262592,0.045053,6.116578,-2.494863,-0.095362,3.593207,...,4.399215,-6.437416,-1.861691,-2.343506,2.913941,-1.396450,-4.003082,2.998931,sub-87964717,SMC
1,0.236989,-4.133460,5.079623,0.865546,-3.095911,-0.385941,5.984298,-2.627577,0.055640,4.088409,...,4.365357,-6.530960,-2.291907,-2.314245,3.359041,-1.046752,-3.754864,3.266617,sub-87964717,SMC
2,0.102437,-3.943042,4.691455,0.775428,-3.469633,-0.194910,6.114291,-2.393794,0.174842,4.567888,...,4.589245,-6.064065,-2.038676,-2.673011,3.074100,-2.738287,-3.435905,2.904586,sub-87964717,SMC


quick SVM model
[[23  9 51 11 14]
 [ 7  9 17 52 23]
 [ 2  7  4 63 32]
 [ 6 16  8 47 31]
 [ 8  7 46 18 29]]
              precision    recall  f1-score   support

        ADHD       0.50      0.21      0.30       108
     HEALTHY       0.19      0.08      0.12       108
         MDD       0.03      0.04      0.03       108
         OCD       0.25      0.44      0.31       108
         SMC       0.22      0.27      0.24       108

    accuracy                           0.21       540
   macro avg       0.24      0.21      0.20       540
weighted avg       0.24      0.21      0.20       540

0.20147619151471657

quick random forest model
[[38 12 37  8 13]
 [17 30 17 26 18]
 [14 29  7 31 27]
 [18 28 11 24 27]
 [19 14 37 14 24]]
              precision    recall  f1-score   support

        ADHD       0.36      0.35      0.36       108
     HEALTHY       0.27      0.28      0.27       108
         MDD       0.06      0.06      0.06       108
         OCD       0.23      0.22      0.23      

In [15]:
# overtrained model
get_ssl_features('fullytrained_acrossRP_tpos2_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-3.531905,4.432943,4.707969,3.826214,-2.983365,1.103101,-5.803141,3.950916,3.165566,-8.639513,...,-0.395701,7.060055,-0.721743,2.165002,4.867585,-5.456316,2.961506,-0.263972,sub-87964717,SMC
1,-4.337537,-2.680801,2.235134,1.783099,-0.681430,5.246514,3.684344,0.435440,-0.621696,-7.269584,...,-3.653046,-0.475181,3.731044,-1.496795,-1.215994,-8.927711,1.810796,-5.170077,sub-87964717,SMC
2,-3.376853,1.623019,-3.751271,3.999388,-2.960108,4.129843,-2.698852,-5.180898,5.402234,-9.942265,...,9.338744,-0.189169,2.992206,-0.020448,0.237041,-11.203195,0.978644,-1.025121,sub-87964717,SMC


quick SVM model
[[58 12 16  6 16]
 [28 26 14 25 15]
 [11 11  7 37 42]
 [16  9 13 48 22]
 [10 10 43 10 35]]
              precision    recall  f1-score   support

        ADHD       0.47      0.54      0.50       108
     HEALTHY       0.38      0.24      0.30       108
         MDD       0.08      0.06      0.07       108
         OCD       0.38      0.44      0.41       108
         SMC       0.27      0.32      0.29       108

    accuracy                           0.32       540
   macro avg       0.32      0.32      0.31       540
weighted avg       0.32      0.32      0.31       540

0.31432896924556275

quick random forest model
[[51 19 13  8 17]
 [23 19 17 27 22]
 [12 16 12 29 39]
 [12 16 19 42 19]
 [15  9 39 16 29]]
              precision    recall  f1-score   support

        ADHD       0.45      0.47      0.46       108
     HEALTHY       0.24      0.18      0.20       108
         MDD       0.12      0.11      0.12       108
         OCD       0.34      0.39      0.37      

#### 2c. Tpos = 4

In [16]:
# best model checkpoint
get_ssl_features('acrossRP_tpos4_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-1.618930,-0.661374,6.977656,-1.921633,-3.223528,-0.738966,0.119964,1.404155,-1.873214,-2.157730,...,0.669622,0.085868,-4.334496,-9.551363,-6.544761,3.049030,-3.395038,0.661277,sub-87964717,SMC
1,-1.198284,-0.923446,7.263509,-2.492660,-3.227293,-0.403087,0.137311,1.589315,-1.659584,-1.710641,...,1.013305,0.208313,-4.383593,-9.946904,-7.222858,3.111385,-2.924509,0.612755,sub-87964717,SMC
2,-1.597099,-0.288571,6.270364,-2.211136,-3.074858,-0.266796,1.228119,1.821659,-2.490983,-1.655024,...,1.349072,0.383931,-4.087228,-9.189544,-6.725908,3.206430,-2.667616,0.578176,sub-87964717,SMC


quick SVM model
[[51 21 23 12  1]
 [ 6 18 18 46 20]
 [ 4  7  9 50 38]
 [ 4 16 18 48 22]
 [ 7  4 51 13 33]]
              precision    recall  f1-score   support

        ADHD       0.71      0.47      0.57       108
     HEALTHY       0.27      0.17      0.21       108
         MDD       0.08      0.08      0.08       108
         OCD       0.28      0.44      0.35       108
         SMC       0.29      0.31      0.30       108

    accuracy                           0.29       540
   macro avg       0.33      0.29      0.30       540
weighted avg       0.33      0.29      0.30       540

0.2993452133970075

quick random forest model
[[59 16 24  8  1]
 [21 32 11 25 19]
 [ 7 26 16 25 34]
 [12 31 15 30 20]
 [18  9 40 13 28]]
              precision    recall  f1-score   support

        ADHD       0.50      0.55      0.52       108
     HEALTHY       0.28      0.30      0.29       108
         MDD       0.15      0.15      0.15       108
         OCD       0.30      0.28      0.29       

In [17]:
# overtrained model
get_ssl_features('fullytrained_acrossRP_tpos4_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-6.269883,3.425868,-0.843336,0.444809,-2.118798,-17.781403,-1.659956,-8.300159,-0.699309,4.597044,...,16.441483,-7.486964,7.381815,-15.652328,-0.892499,-0.386531,2.221758,5.009332,sub-87964717,SMC
1,2.052338,-0.867050,2.808499,-0.249947,4.628599,-2.310798,-6.486875,-3.759819,-1.997615,1.098362,...,-7.461098,-11.164515,-4.003872,-8.552828,-2.325557,2.772589,-6.005247,-2.143929,sub-87964717,SMC
2,-6.528491,3.434939,6.315592,8.677110,-0.120331,-3.469843,-6.559155,2.544773,-8.217282,6.129962,...,-8.247609,-14.891423,-9.117801,-13.865042,-4.268693,3.318489,3.804268,5.996053,sub-87964717,SMC


quick SVM model
[[51  9 17 10 21]
 [25 25 18 25 15]
 [ 8 17 12 38 33]
 [17 17 12 49 13]
 [11  9 29 20 39]]
              precision    recall  f1-score   support

        ADHD       0.46      0.47      0.46       108
     HEALTHY       0.32      0.23      0.27       108
         MDD       0.14      0.11      0.12       108
         OCD       0.35      0.45      0.39       108
         SMC       0.32      0.36      0.34       108

    accuracy                           0.33       540
   macro avg       0.32      0.33      0.32       540
weighted avg       0.32      0.33      0.32       540

0.31779339344205226

quick random forest model
[[52  9 15  7 25]
 [25 27 17 19 20]
 [15 17 13 35 28]
 [18 19 18 43 10]
 [11 15 40 20 22]]
              precision    recall  f1-score   support

        ADHD       0.43      0.48      0.45       108
     HEALTHY       0.31      0.25      0.28       108
         MDD       0.13      0.12      0.12       108
         OCD       0.35      0.40      0.37      

### 3. Across Subject pretext task

In [18]:
# best model checkpoint
get_ssl_features('acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-0.813289,-2.716501,0.919075,0.711839,-0.579521,-0.587127,-1.335802,0.153440,-0.412433,-2.323945,...,0.507714,2.636543,0.503704,0.827685,0.395858,0.120876,-0.415438,-0.451464,sub-87964717,SMC
1,-0.504195,-2.467970,1.108659,1.404283,0.195515,-2.225784,-2.613005,-0.281208,-0.971093,-2.152495,...,-1.801580,3.331660,1.010935,1.049690,2.478003,0.066149,-0.315350,-0.660595,sub-87964717,SMC
2,-0.830035,-2.191188,0.291836,1.731114,0.140483,-1.779407,-1.535045,-0.424333,0.150285,-1.432060,...,0.596767,2.368657,-0.707786,0.529840,1.082571,-0.320324,0.054474,-0.413685,sub-87964717,SMC


quick SVM model
[[55 12 12 24  5]
 [ 1 50  2 40 15]
 [ 0 36 16  8 48]
 [ 3 39 13 37 16]
 [ 0 18 26 11 53]]
              precision    recall  f1-score   support

        ADHD       0.93      0.51      0.66       108
     HEALTHY       0.32      0.46      0.38       108
         MDD       0.23      0.15      0.18       108
         OCD       0.31      0.34      0.32       108
         SMC       0.39      0.49      0.43       108

    accuracy                           0.39       540
   macro avg       0.44      0.39      0.40       540
weighted avg       0.44      0.39      0.40       540

0.3953832393595814

quick random forest model
[[57 12  6 30  3]
 [ 3 44 12 42  7]
 [ 7 27 27  8 39]
 [11 30 26 26 15]
 [ 5 15 17 15 56]]
              precision    recall  f1-score   support

        ADHD       0.69      0.53      0.60       108
     HEALTHY       0.34      0.41      0.37       108
         MDD       0.31      0.25      0.28       108
         OCD       0.21      0.24      0.23       

In [29]:
# overtrained model
get_ssl_features('fullytrained_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-33.949890,-21.187607,40.104279,4.188790,-12.426004,-3.792988,-6.778365,3.670086,-8.454637,-12.776423,...,9.429151,48.784637,-5.315949,17.138603,8.304023,-19.232035,20.425667,-27.241877,sub-87964717,SMC
1,-35.625240,-20.766567,44.157349,2.638543,-7.202231,3.174234,-19.622866,-10.128608,-2.209416,-7.286963,...,-3.001688,52.417896,-4.882135,18.492725,-2.724310,-12.108669,22.986965,-19.918627,sub-87964717,SMC
2,-36.063278,-19.276077,38.083809,-1.578408,2.942086,-7.001598,-11.447214,0.169034,3.982788,-9.969617,...,12.822247,45.334496,-4.750672,25.247160,6.399560,-13.530219,19.931404,-23.771576,sub-87964717,SMC


quick SVM model
[[60  0 32 14  2]
 [12 53  3 40  0]
 [ 7 23 21 34 23]
 [45  8 12 42  1]
 [ 3 18 32  0 55]]
              precision    recall  f1-score   support

        ADHD       0.47      0.56      0.51       108
     HEALTHY       0.52      0.49      0.50       108
         MDD       0.21      0.19      0.20       108
         OCD       0.32      0.39      0.35       108
         SMC       0.68      0.51      0.58       108

    accuracy                           0.43       540
   macro avg       0.44      0.43      0.43       540
weighted avg       0.44      0.43      0.43       540

0.43045500760769845

quick random forest model
[[62  3 28 13  2]
 [ 4 63  1 40  0]
 [ 4 20 14 41 29]
 [30 24 15 35  4]
 [ 1  8 45  0 54]]
              precision    recall  f1-score   support

        ADHD       0.61      0.57      0.59       108
     HEALTHY       0.53      0.58      0.56       108
         MDD       0.14      0.13      0.13       108
         OCD       0.27      0.32      0.30      

#### Test min loss

In [7]:
# best model checkpoint
get_ssl_features('v2_150feat_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet', num_extracted_features=150)

features_df.shape = (2700, 152)


,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,ID,diagnosis
0,1.647571,-2.053078,-4.633385,2.251067,-5.850496,0.025109,5.551583,2.534439,-0.778269,0.462926,...,-0.629383,1.386664,1.477166,0.523306,0.025430,-0.342440,3.620466,-10.023107,sub-87964717,SMC
1,2.989523,-0.448032,-7.620080,2.511840,-4.813479,0.145670,6.631098,2.870703,-0.881683,-0.395096,...,1.507385,2.120140,3.632714,0.505810,1.521411,10.999740,-1.445104,-12.330227,sub-87964717,SMC
2,1.991210,-2.403431,-8.553134,3.775620,-7.334594,-1.191084,6.183104,2.793301,-1.420995,1.264022,...,-0.202577,0.746111,1.889090,0.932629,-0.256928,-1.186927,-5.554991,-9.398594,sub-87964717,SMC


quick SVM model
[[48  6 23 24  7]
 [ 9 57  8 33  1]
 [ 5 19 20 37 27]
 [25 15 33 24 11]
 [ 3 19 24 11 51]]
              precision    recall  f1-score   support

        ADHD       0.53      0.44      0.48       108
     HEALTHY       0.49      0.53      0.51       108
         MDD       0.19      0.19      0.19       108
         OCD       0.19      0.22      0.20       108
         SMC       0.53      0.47      0.50       108

    accuracy                           0.37       540
   macro avg       0.38      0.37      0.38       540
weighted avg       0.38      0.37      0.38       540

0.3758109725283235

quick random forest model
[[58  9 22 13  6]
 [ 3 52 14 35  4]
 [14 21 16 28 29]
 [27 22 30 24  5]
 [ 2 13 48  5 40]]
              precision    recall  f1-score   support

        ADHD       0.56      0.54      0.55       108
     HEALTHY       0.44      0.48      0.46       108
         MDD       0.12      0.15      0.13       108
         OCD       0.23      0.22      0.23       

In [8]:
# overtrained model
get_ssl_features('v2_fullytrained_150feat_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet', num_extracted_features=150) 

features_df.shape = (2700, 152)


,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,ID,diagnosis
0,-10.801856,4.725545,-14.806151,2.223729,-10.305056,4.926569,-6.564208,6.813509,2.426950,12.543073,...,-0.718361,8.038696,10.914590,25.360149,-7.751112,-9.054343,3.188895,-26.129961,sub-87964717,SMC
1,-7.688347,8.796160,-24.405148,3.251776,-12.371649,3.810916,-5.133356,4.658499,1.788880,6.762197,...,-0.590510,7.722211,8.072158,29.228466,0.605801,9.804248,-3.704033,-33.334587,sub-87964717,SMC
2,-2.251540,3.369425,-22.512278,4.836353,-12.423389,5.270777,-5.184250,4.780623,-3.137332,14.966404,...,-4.240508,7.230493,1.850020,28.166121,1.592426,-3.042860,-5.274507,-16.146692,sub-87964717,SMC


quick SVM model
[[58  0 26 23  1]
 [12 54  4 38  0]
 [ 4 26 28 32 18]
 [36 10 23 37  2]
 [ 1 20 35  1 51]]
              precision    recall  f1-score   support

        ADHD       0.52      0.54      0.53       108
     HEALTHY       0.49      0.50      0.50       108
         MDD       0.24      0.26      0.25       108
         OCD       0.28      0.34      0.31       108
         SMC       0.71      0.47      0.57       108

    accuracy                           0.42       540
   macro avg       0.45      0.42      0.43       540
weighted avg       0.45      0.42      0.43       540

0.4302766613925021

quick random forest model
[[60  2 30 15  1]
 [ 6 57  6 39  0]
 [ 9 22 19 28 30]
 [22 20 25 40  1]
 [ 0 15 41  3 49]]
              precision    recall  f1-score   support

        ADHD       0.62      0.56      0.59       108
     HEALTHY       0.49      0.53      0.51       108
         MDD       0.16      0.18      0.17       108
         OCD       0.32      0.37      0.34       

In [6]:
# best model checkpoint
get_ssl_features('v2_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,7.646394,-8.162047,-0.250230,1.359651,11.568721,-0.345870,0.070589,2.746930,1.451071,-8.297832,...,5.851769,1.647453,-5.214857,-3.316376,0.023578,-57.526985,-2.273098,2.032529,sub-87964717,SMC
1,6.767371,-10.118893,-1.974311,4.377435,9.784183,-0.863558,1.450167,0.444023,-0.488292,-8.604809,...,13.597450,2.242329,-5.312629,-3.650613,-7.799878,-57.248699,-0.119202,5.252287,sub-87964717,SMC
2,6.738260,-8.012308,-0.815649,2.126563,9.427182,-1.606308,0.271904,1.505939,0.892188,-8.437885,...,12.368105,1.185365,-4.702768,-3.509149,1.591986,-59.196247,-0.941873,1.603270,sub-87964717,SMC


quick SVM model
[[57  6 14 26  5]
 [12 56 11 28  1]
 [12 15 15 37 29]
 [43 10 27 25  3]
 [ 4 18 38  3 45]]
              precision    recall  f1-score   support

        ADHD       0.45      0.53      0.48       108
     HEALTHY       0.53      0.52      0.53       108
         MDD       0.14      0.14      0.14       108
         OCD       0.21      0.23      0.22       108
         SMC       0.54      0.42      0.47       108

    accuracy                           0.37       540
   macro avg       0.37      0.37      0.37       540
weighted avg       0.37      0.37      0.37       540

0.3682372039573172

quick random forest model
[[60  9 19 19  1]
 [12 47 11 33  5]
 [ 9 20 18 35 26]
 [28 20 35 21  4]
 [ 2 12 60  3 31]]
              precision    recall  f1-score   support

        ADHD       0.54      0.56      0.55       108
     HEALTHY       0.44      0.44      0.44       108
         MDD       0.13      0.17      0.14       108
         OCD       0.19      0.19      0.19       

In [7]:
# overtrained model
get_ssl_features('v2_fullytrained_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet') 

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,14.436729,1.926414,7.237008,1.524072,34.271229,-22.272837,14.027569,15.874889,-1.317448,-24.636742,...,13.067459,0.528617,-16.241306,34.755054,8.339795,-76.853355,3.412999,10.248852,sub-87964717,SMC
1,15.706059,1.841708,5.456678,8.480411,31.147490,-23.194834,21.043114,13.146032,-2.210413,-24.139063,...,20.911539,1.329855,-11.290009,41.607433,-3.721012,-78.685196,3.321112,17.304354,sub-87964717,SMC
2,10.957920,1.902343,8.038800,5.854018,32.850132,-22.969179,19.862272,14.692191,-0.507492,-26.714600,...,18.294111,-3.513967,-11.163970,34.639332,13.450257,-79.795242,4.173875,15.663052,sub-87964717,SMC


quick SVM model
[[57  0 37 13  1]
 [12 39 13 44  0]
 [13 21 18 32 24]
 [48  9 21 30  0]
 [ 3 13 32  1 59]]
              precision    recall  f1-score   support

        ADHD       0.43      0.53      0.47       108
     HEALTHY       0.48      0.36      0.41       108
         MDD       0.15      0.17      0.16       108
         OCD       0.25      0.28      0.26       108
         SMC       0.70      0.55      0.61       108

    accuracy                           0.38       540
   macro avg       0.40      0.38      0.38       540
weighted avg       0.40      0.38      0.38       540

0.38370036593549506

quick random forest model
[[61  3 28 16  0]
 [ 7 47  6 47  1]
 [15 22 12 32 27]
 [26 23 21 33  5]
 [ 0  8 46  3 51]]
              precision    recall  f1-score   support

        ADHD       0.56      0.56      0.56       108
     HEALTHY       0.46      0.44      0.45       108
         MDD       0.11      0.11      0.11       108
         OCD       0.25      0.31      0.28      

### 4. Contrastive Loss pretext task

#### 4a. trained for 3000 epochs

In [ ]:
# best model checkpoint
get_ssl_features('subsaug25f_contrastive_loss_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,1.044557,4.492836,-3.806807,-0.768025,0.937985,2.615165,3.092052,3.132883,-3.854393,-0.937764,...,-2.703764,-3.290643,-0.366950,-3.882891,-3.543538,-3.786275,2.405729,-4.072068,sub-87964717,SMC
1,0.702214,3.874519,-3.824599,0.110002,0.566617,3.056918,3.931499,3.736588,-3.271497,-0.577326,...,-3.454946,-4.204899,-1.913187,-4.381532,-4.482499,-4.507197,2.515676,-5.366399,sub-87964717,SMC
2,0.550182,2.792593,-3.291342,-0.424154,-0.161782,2.861225,2.591897,2.830465,-2.609921,0.398929,...,-2.663291,-3.430825,-1.134340,-2.801837,-3.489840,-3.990401,1.524168,-4.083732,sub-87964717,SMC


quick SVM model
[[61  6 12 22  7]
 [ 0 18 11 38 41]
 [ 0  5  1 42 60]
 [ 0 14 17 34 43]
 [ 6 12 25 26 39]]
              precision    recall  f1-score   support

        ADHD       0.91      0.56      0.70       108
     HEALTHY       0.33      0.17      0.22       108
         MDD       0.02      0.01      0.01       108
         OCD       0.21      0.31      0.25       108
         SMC       0.21      0.36      0.26       108

    accuracy                           0.28       540
   macro avg       0.33      0.28      0.29       540
weighted avg       0.33      0.28      0.29       540

0.28861856480334935

quick random forest model
[[72 13  4 14  5]
 [10 32 18 29 19]
 [ 8 16 13 24 47]
 [12 30 15 21 30]
 [12 20 25 19 32]]
              precision    recall  f1-score   support

        ADHD       0.63      0.67      0.65       108
     HEALTHY       0.29      0.30      0.29       108
         MDD       0.17      0.12      0.14       108
         OCD       0.20      0.19      0.20      

In [ ]:
# overtrained model
get_ssl_features('fullytrained_subsaug25f_contrastive_loss_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,10.831122,-6.431435,3.565138,-4.058208,1.358812,1.883469,13.823512,-0.096010,-4.172929,1.480669,...,5.255815,-0.360995,3.309501,5.053679,-2.028048,-3.017830,0.498548,-1.036240,sub-87964717,SMC
1,10.969341,-3.782219,-5.193200,1.434877,4.193954,9.342376,6.060780,6.263244,-3.191915,3.335055,...,-1.438563,-4.139843,4.331742,-2.005771,6.573871,1.823699,-6.486544,-0.192590,sub-87964717,SMC
2,3.181119,-0.934810,-3.809564,-0.214173,0.907642,6.974266,3.782580,5.713600,4.304003,-1.448043,...,3.258717,4.257723,11.273497,3.521773,-2.313400,-0.978868,-2.289573,-1.362508,sub-87964717,SMC


quick SVM model
[[48 25 15 18  2]
 [11 44 14 25 14]
 [ 3 35 13 22 35]
 [ 9 47  9 32 11]
 [ 6 11 19  8 64]]
              precision    recall  f1-score   support

        ADHD       0.62      0.44      0.52       108
     HEALTHY       0.27      0.41      0.33       108
         MDD       0.19      0.12      0.15       108
         OCD       0.30      0.30      0.30       108
         SMC       0.51      0.59      0.55       108

    accuracy                           0.37       540
   macro avg       0.38      0.37      0.37       540
weighted avg       0.38      0.37      0.37       540

0.3676780582303608

quick random forest model
[[55 25 13 12  3]
 [16 51 12 15 14]
 [ 1 32 15 19 41]
 [ 4 47 13 27 17]
 [ 5 16 14 10 63]]
              precision    recall  f1-score   support

        ADHD       0.68      0.51      0.58       108
     HEALTHY       0.30      0.47      0.37       108
         MDD       0.22      0.14      0.17       108
         OCD       0.33      0.25      0.28       